# Filtracja Non-Local Means

## Definicja

Kolejny "poziom wtajemniczenia" w zagadnienie filtracji obrazów to metoda Non-Local Means (NLM).
Została ona zaproponowana w pracy *A non-local algorithm for image denoising* autorstwa Antoni Buades, Bartomeu Coll, i Jean Michel Morel na konferencji CVPR w 2005 roku.

Filtr NLM dany jest zależnością:

\begin{equation}
\hat{I}(\mathbf{x}) = \sum_{\mathbf{p} \in V(\mathbf{x})} w(\mathbf{p},\mathbf{x})I(\mathbf{p})
\end{equation}

gdzie:
- $I$ - obraz wejściowy,
- $\hat{I}$ - obraz wyjściowy (przefiltrowany),
- $\mathbf{x}$ - współrzędne piksela obrazu,
- $V(\mathbf{x})$ - obszar poszukiwań piksela, dla którego przeprowadzana jest filtracja,
- $w$ - waga punktu $\mathbf{p}$ z obszaru poszukiwań.

Wróćmy na chwilę do filtracji bilateralnej. Tam waga danego piksela z kontekstu zależała od dwóch czynników - odległości przestrzennej pomiędzy pikselami oraz różnicy w jasności/kolorze pomiędzy pikselami (tzw. przeciwdziedzina).
Filtr NLM stanowi uogólnienie tej metody - do obliczania wag nie wykorzystuje się już pojedynczych pikseli ($\mathbf{p}$ i $\mathbf{x}$), a lokalne konteksty ($N(\mathbf{p})$ i $N(\mathbf{x})$).

Waga $w$ dana jest następującą zależnością:

\begin{equation}
w(\mathbf{p},\mathbf{x}) = \frac{1}{Z(\mathbf{x})}\exp(-\frac{|| v(N(\mathbf{p})) - v(N(\mathbf{x})) ||^2_{2}}{\alpha \sigma^2})
\end{equation}

gdzie:
- $Z(\mathbf{x}) = \sum_{\mathbf{p} \in  V(\mathbf{x})} \exp(-\frac{|| v(N(\mathbf{p})) - v(N(\mathbf{x})) ||^2_{2}}{\alpha \sigma^2})$,
- $|| \cdot ||$ - jest normą $L_2$ odległości pomiędzy dwoma kontekstami,
- $v$ oznacza mnożenie punktowe kontekstu $N$ przez dwuwymiarową maskę Gaussa o odpowiadających kontekstowi wymiarach,
- $\alpha$ > 0 - parametr sterujący filtracją,
- $\sigma$ - parametr szumu stacjonarnego występującego na obrazie (w przypadku szumu niestacjonarnego, parametr $\sigma$ musi zostać dopasowany lokalnie tj. $\sigma = \sigma(\mathbf{x})$).

## Analiza działania

Zastanówmy sie teraz jak działa filtra NLM. Najprościej to zrozumieć na rysunku.

![Ilustracja NLM](https://raw.githubusercontent.com/vision-agh/poc_sw/master/07_Bilateral/nlm.png)

1. Dla rozważanego piksela $\mathbf{x}$ definiujemy obszar poszukiwań $V(\mathbf{x})$. Uwaga - obszar poszukiwań ($V$) jest jednostką większą niż otocznie/kontekst ($N$).

2. Następnie, dla każdego z pikseli $\mathbf{p} \in  V(\mathbf{x})$ oraz samego $\mathbf{x}$ definiujemy otocznie/kontekst odpowiednio $N(\mathbf{p})$ i $N(\mathbf{x})$.

3. Wracamy do równania definiującego wagę  $w(\mathbf{p},\mathbf{x})$, a konkretnie do wyrażenia $|| v(N(\mathbf{p})) - v(N(\mathbf{x})) ||$. Przeanalizujmy co ono oznacza. Mamy dwa otoczenia: $N(\mathbf{p})$ i $N(\mathbf{x})$. Każde z nich mnożymy przez odpowiadającą maskę Gaussa - funkcja $v$. Otrzymujemy dwie macierze, które odejmujemy od siebie punktowo. Następnie obliczamy kwadrat z normy ($L_2$ definiujemy jako $||X||_2 = \sqrt{\sum_k|X_k|^2}$. Otrzymujemy zatem jedną liczbę, która opisuje nam podobieństwo otoczeń pikseli $\mathbf{x}$ i $\mathbf{p}$. Mała wartość oznacza otoczenia zbliżone, duża - różniące się. Ponieważ, z dokładnością do stałych, liczba ta stanowi wykładnik funkcji $e^{-x}$, to ostatecznie waga jest zbliżona do 1 dla otoczeń podobnych, a szybko maleje wraz z malejącym podobieństwem kontekstów.

4. Podsumowując. Jak wynika z powyższej analizy filtr NLM to taki filtr bilateralny, w którym zamiast pojedynczych pikseli porównuje się ich lokalne otoczenia. Wpływa to pozytywnie na jakość filtracji, niestety kosztem złożoności obliczeniowej.

In [ ]:
import cv2
import os
import requests
from matplotlib import pyplot as plt
import numpy as np
from scipy import signal
from scipy.io import loadmat
import math

url = 'https://raw.githubusercontent.com/vision-agh/poc_sw/master/07_Bilateral/'

fileNames = ["MR_data.mat"]
for fileName in fileNames:
    if not os.path.exists(fileName):
        r = requests.get(url + fileName, allow_redirects=True)
        open(fileName, 'wb').write(r.content)

In [ ]:
def compare_imgs(imgs, titles=None):
    _, axs = plt.subplots(1, len(imgs), figsize=(20, 20))
    for i, img in enumerate(imgs):
        axs[i].imshow(img, 'gray')
        axs[i].set_title(titles[i] if titles else '')
        axs[i].axis('off')
    plt.show()

## Implementacja

W ramach zadania należy zaimplementować filtr NLM, ocenić jego działanie w porównaniu do filtra Gaussa i bilateralnego oraz dokonać pomiaru czasu obliczeń (dla trzech wymienionych metod).

Jak już się zrozumie jak działa NLM, jego implementacja jest dość prosta.
Wartość parametru $\alpha$ należy dobrać eksperymentalnie.
Nie należy także "przesadzić" z rozmiarem obszaru poszukiwań (np. 11x11) oraz kontekstu (5x5 lub 3x3).

Wskazówki do implementacji:
- algorytm sprowadza się do dwóch podwójnych pętli for: zewnętrzne po pikselach, wewnętrzne po kolejnych obszarach przeszukań,
- przed realizacją trzeba przemyśleć problem pikseli brzegowych - de facto problemów jest kilka. Po pierwsze nie dla każdego piksela można wyznaczyć pełny obszar przeszukań (tu propozycja, aby filtrację przeprowadzać tylko dla pikseli z pełnym obszarem). Po drugie, ponieważ rozpatrujemy konteksty, to nawet dla piksela o "pełnym" obszarze przeszukań, będą istnieć piksele, dla których nie pełnych kontekstów (sugestia - powiększyć obszar przeszukać, tak aby zawierał konteksty). Ostatni problem jest bardziej techniczny/implementacyjny. Jeśli w kolejnych iteracjach "jawnie" wytniemy fragment o rozmiarach obszaru przeszukiwań, to znowu pojawi się problem brzegowy - tu można albo wyciąć nieco większy obszar, albo cały czas "pracować" na obrazie oryginalnym ("żonglerka indeksami").
- warto sprawdzać indeksy i rozmiary "wycinanych" kontekstów,
- wagi wyliczamy w trzech krokach:
    - obliczenia dla $N(\mathbf{x})$ + inicjalizacja macierzy na wagi,
    - podwójna pętla, w której przeprowadzamy obliczenia dla kolejnych $N(\mathbf{p})$ oraz wyliczamy wagi,
    - normalizacja macierzy wag oraz końcowa filtracja obszaru w wykorzystaniem wag.
- uwaga, obliczenia trochę trwają, nawet dla obrazka 256x256 i względnie niewielkich obszaru przeszukań i kontesktu.

Efekt końcowy:
- porównanie wyników metod: filtr Gaussa, filtr bilateralny oraz filtr NLM (2-3 zdania komentarza),
- porównanie czasu działania powyższych metod (1 zdanie komentarza).


In [ ]:
mat = loadmat('MR_data.mat')

In [ ]:
img_names = ['I_noisy1', 'I_noisy2', 'I_noisy3', 'I_noisy4']
imgs = [mat[img_name] for img_name in img_names]

In [ ]:
def mesh(fun):
    """Funkcja do wizualizacji filtra gausowskiego"""
    fig = plt.figure()
    ax = fig.add_subplot(projection = '3d')

    size = int(fun.shape[0]//2)
    X = np.arange(-size, size+1, 1)
    Y = np.arange(-size, size+1, 1)
    X, Y = np.meshgrid(X, Y)
    Z = fun

    ax.plot_surface(X, Y, Z)
    plt.show()

def fgaussian(size, sigma):
    """Funkcja do obliczania współczynnika filtra Gaussowskiego"""
    m = n = size
    h, k = m//2, n//2
    x, y = np.mgrid[-h : h+1, -k : k+1]
    g = np.exp(-(x**2 + y**2)/(2 * sigma**2))
    return g / g.sum()

def gamma(y, delta_r):
    """Funkcja do obliczania współczynnika funkcji odległości w przeciwdziedzinie obrazu"""
    return np.exp(-y ** 2 / (2 * delta_r**2))

In [ ]:
def gaussian_filter(img, size_, delta_s):
    """Implementacja filtracji Gaussowskiej"""
    size = int(size_//2)
    img_filtered = np.zeros_like(img)
    
    gaussian_filter = fgaussian(size_, delta_s)
    # mesh(gaussian_filter)
    
    def get_new_value(x, y):
        window = img[x-size: x + size+1, y-size: y + size+1]
        return np.sum(window * gaussian_filter)

    for x in range(size, img_filtered.shape[0]-size):
        for y in range(size, img_filtered.shape[1]-size):
            img_filtered[x, y] = get_new_value(x, y)

    return np.uint8(img_filtered)

In [ ]:
def bilateral_filter(img, size_, delta_s, delta_r):
    """Implementacja filtrowania bilateralnego"""
    size = int(size_//2)
    img_filtered = np.zeros_like(img)
    
    gaussian_filter = fgaussian(size_, delta_s)
    # mesh(gaussian_filter)

    def get_new_value(x, y):
        window = img[x - size: x + size + 1, y - size: y + size + 1]

        filter_window = gaussian_filter * gamma(np.abs(window - img[x, y]), delta_r)
        normalized_filter = filter_window / np.sum(filter_window)

        return np.sum(window * normalized_filter)
    
    for x in range(size, img_filtered.shape[0]-size):
        for y in range(size, img_filtered.shape[1]-size):
            img_filtered[x, y] = get_new_value(x, y)
            

    return np.uint8(img_filtered)

In [ ]:
def non_local_means_filter(img, search_area_size, patch_size, alpha, sigma, delta):
    height, width = img.shape
    size_ = int(patch_size//2)
    area_ = int(search_area_size//2)
    img_filtered = np.zeros_like(img)
    
    gaussian_mask = fgaussian(patch_size, delta)
    
    def neighbourhood(x, y, s=size_):
        return img[x-s : x+s+1, y-s : y+s+1]
    
    def weight(nei_x, nei_p):
        nei_x = gaussian_mask * nei_x
        nei_p = gaussian_mask * nei_p
        return math.exp(- np.sum((nei_p - nei_x)**2) / (alpha * sigma**2))
    
    def get_new_value(x, y):
        x_min = max(size_    , x-area_           )
        x_max = min(x+area_+1, height-area_-size_)
        y_min = max(size_    , y-area_           )
        y_max = min(y+area_+1, width-area_-size_ )
    
        window = img[x_min:x_max, y_min:y_max]
        weights = np.zeros_like(window)
        nei_x = neighbourhood(x, y)
        
        for i in range(weights.shape[0]):
            for j in range(weights.shape[1]):
                nei_p = neighbourhood(i+x, j+y)
                weights[i, j] = weight(nei_x, nei_p)
        norm_weights = weights / np.sum(weights)
        
        return np.sum(norm_weights * window)
    
    for x in range(size_, height-size_):
        for y in range(size_, width-size_):
            img_filtered[x, y] = get_new_value(x, y)
            
    return img_filtered        

In [ ]:
import time


# Wczytanie obrazu
img = imgs[0]

# Parametry filtrów
spatial_sigma = 1.0
range_sigma = 20.0
search_area_size = 7
patch_size = 3
alpha = 0.1
sigma = 10.0

# Filtracja Gaussa
start_time = time.time()
gaussian_result = gaussian_filter(img, patch_size, spatial_sigma)
gaussian_time = time.time() - start_time

# Filtracja bilateralna
start_time = time.time()
bilateral_result = bilateral_filter(img, patch_size, spatial_sigma, range_sigma)
bilateral_time = time.time() - start_time

# Filtracja Non-Local Means
start_time = time.time()
nlm_result = non_local_means_filter(img, search_area_size, patch_size, alpha, sigma, spatial_sigma)
nlm_time = time.time() - start_time

# Wyświetlenie wyników
compare_imgs([img, gaussian_result, bilateral_result, nlm_result], ['Original Image', 'Gaussian Filter\nTime: {:.4f}s'.format(gaussian_time), 'Bilateral Filter\nTime: {:.4f}s'.format(bilateral_time), 'Non-Local Means Filter\nTime: {:.4f}s'.format(nlm_time)])